In [ ]:
import os
import hashlib
from collections import defaultdict
from tqdm import tqdm  # 진행상황 표시용 (없으면 pip install tqdm)

def get_file_hash(filepath):
    """파일 내용을 읽어서 MD5 해시값을 반환 (이미지 내용 비교용)"""
    hasher = hashlib.md5()
    with open(filepath, 'rb') as f:
        buf = f.read()
        hasher.update(buf)
    return hasher.hexdigest()

def analyze_exo_duplicates(root_dir):
    # 데이터 저장소: { 이미지해시: { 'actions': {set}, 'paths': [] } }
    image_db = defaultdict(lambda: {'actions': set(), 'objects': set(), 'paths': []})
    
    total_files_count = 0
    
    print(f"Scanning directory: {root_dir} ...")
    
    # 1. 모든 폴더 순회
    # 구조: root/action/object/image.jpg
    for action in os.listdir(root_dir):
        action_path = os.path.join(root_dir, action)
        if not os.path.isdir(action_path):
            continue
            
        for obj in os.listdir(action_path):
            obj_path = os.path.join(action_path, obj)
            if not os.path.isdir(obj_path):
                continue
                
            # 이미지 파일 스캔
            files = [f for f in os.listdir(obj_path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
            
            for file in files:
                full_path = os.path.join(obj_path, file)
                total_files_count += 1
                
                # 파일 해시 계산 (내용 기반 중복 체크)
                # 속도가 느리면 파일명(file)으로 대체 가능하지만, 해시가 가장 정확함
                img_hash = get_file_hash(full_path)
                
                image_db[img_hash]['actions'].add(action)
                image_db[img_hash]['objects'].add(obj)
                image_db[img_hash]['paths'].append(full_path)

    # 2. 통계 계산
    unique_images_count = len(image_db)
    
    # 여러 액션에 동시에 속한 이미지 찾기 (교집합)
    overlapping_images = 0
    overlapping_details = []

    for img_hash, info in image_db.items():
        if len(info['actions']) > 1:
            overlapping_images += 1
            overlapping_details.append({
                'actions': list(info['actions']),
                'object': list(info['objects']),
                'count': len(info['actions']),
                'sample_path': info['paths'][0]
            })

    # 3. 결과 출력
    print("\n" + "="*50)
    print("📊 Exocentric Image Statistics Analysis")
    print("="*50)
    
    print(f"1) Total Image Files (Raw Count)   : {total_files_count:,}")
    print(f"2) Unique Images (Content Based)   : {unique_images_count:,}")
    
    overlap_percent = (overlapping_images / unique_images_count) * 100 if unique_images_count > 0 else 0
    print(f"3) Overlapping Images (Multi-Action): {overlapping_images:,} ({overlap_percent:.2f}%)")
    
    print("\n[Analysis Result]")
    print(f"- Out of {unique_images_count} unique images, {overlapping_images} images appear in multiple action categories.")
    print(f"- This confirms that {overlap_percent:.1f}% of the exocentric knowledge is inherently ambiguous (Type-1 Failure).")

    # 4. 겹치는 예시 몇 개 출력 (논문에 쓰기 좋음)
    print("\n[Examples of Overlapping Images]")
    for i, item in enumerate(overlapping_details[:5]):
        print(f"{i+1}. Actions: {item['actions']} (Object: {item['object']})")
        print(f"   File: {os.path.basename(item['sample_path'])}")

# 실행 경로 설정
root_directory = "/home/DATA/AGD20K/Seen/trainset/exocentric"
analyze_exo_duplicates(root_directory)

Scanning directory: /home/DATA/AGD20K/Seen/trainset/exocentric ...

📊 Exocentric Image Statistics Analysis
1) Total Image Files (Raw Count)   : 20,061
2) Unique Images (Content Based)   : 14,092
3) Overlapping Images (Multi-Action): 4,461 (31.66%)

[Analysis Result]
- Out of 14092 unique images, 4461 images appear in multiple action categories.
- This confirms that 31.7% of the exocentric knowledge is inherently ambiguous (Type-1 Failure).

[Examples of Overlapping Images]
1. Actions: ['hold', 'drag'] (Object: ['suitcase'])
   File: drag_suitcase_001628.jpg
2. Actions: ['hold', 'drag'] (Object: ['suitcase'])
   File: drag_suitcase_011004.jpg
3. Actions: ['hold', 'drag'] (Object: ['suitcase'])
   File: drag_suitcase_014039.jpg
4. Actions: ['hold', 'drag'] (Object: ['suitcase'])
   File: drag_suitcase_001094.jpg
5. Actions: ['hold', 'drag'] (Object: ['suitcase'])
   File: drag_suitcase_013081.jpg


In [1]:
import os

def count_images_in_dir(root_dir):
    """디렉토리 내의 모든 이미지 파일(.jpg, .png 등) 개수를 재귀적으로 셉니다."""
    image_count = 0
    
    if not os.path.exists(root_dir):
        print(f"⚠️ Warning: Directory not found -> {root_dir}")
        return 0

    # os.walk를 쓰면 하위 폴더까지 자동으로 다 뒤집니다.
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for file in filenames:
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                image_count += 1
                
    return image_count

# -----------------------------------------------------------
# 설정: 경로 지정
# -----------------------------------------------------------
base_path = "/home/DATA/AGD20K"

# 1. Seen Testset 경로
seen_test_ego = os.path.join(base_path, "Seen/testset/egocentric")
seen_test_gt = os.path.join(base_path, "Seen/testset/GT") # 혹시 GT 개수도 궁금할까봐 추가

# 2. Unseen Testset 경로
unseen_test_ego = os.path.join(base_path, "Unseen/testset/egocentric")
unseen_test_gt = os.path.join(base_path, "Unseen/testset/GT")

# -----------------------------------------------------------
# 실행 및 출력
# -----------------------------------------------------------
print("="*50)
print("📊 AGD20K Testset Image Counts")
print("="*50)

# Seen Testset 계산
count_seen_ego = count_images_in_dir(seen_test_ego)
count_seen_gt = count_images_in_dir(seen_test_gt)

print(f"✅ [Seen Testset]")
print(f"   - Egocentric Images : {count_seen_ego:,}")
print(f"   - Ground Truth (GT) : {count_seen_gt:,}")

print("-" * 30)

# Unseen Testset 계산
count_unseen_ego = count_images_in_dir(unseen_test_ego)
count_unseen_gt = count_images_in_dir(unseen_test_gt)

print(f"✅ [Unseen Testset]")
print(f"   - Egocentric Images : {count_unseen_ego:,}")
print(f"   - Ground Truth (GT) : {count_unseen_gt:,}")

print("="*50)
print(f"👉 Total Test Images (Seen + Unseen): {count_seen_ego + count_unseen_ego:,}")

📊 AGD20K Testset Image Counts
✅ [Seen Testset]
   - Egocentric Images : 1,710
   - Ground Truth (GT) : 1,675
------------------------------
✅ [Unseen Testset]
   - Egocentric Images : 547
   - Ground Truth (GT) : 540
👉 Total Test Images (Seen + Unseen): 2,257
